* 测试moss关于输入序列长度和GPU显存的关系

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import matplotlib.pyplot as plt 
import numpy as np
import os
import math

/home/qs/anaconda3/envs/moss/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# os.environ["CUDA_LAUNCH_BLOCKING"]="1"
# model = AutoModelForCausalLM.from_pretrained("fnlp/moss-moon-003-sft-int4", trust_remote_code=True).half().cuda("cuda:1")
model = AutoModelForCausalLM.from_pretrained("../model/moss-moon-003-sft-int8", trust_remote_code=True).half().cuda("cuda:0")
model_size = torch.cuda.memory_allocated()/1024/1024/1024
print("模型占用显存（G）：",model_size)
# model.eval()

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


模型占用显存（G）： 17.27969455718994


In [ ]:
x, y =[], []
for i in range(64, 2048, 32):
    torch.cuda.empty_cache()
    try:
        demo_data = torch.zeros((1,i),dtype=torch.long).to("cuda:0")
        _=model(demo_data) 
        x.append(i)
        # data_size = torch.cuda.memory_allocated('cuda:0')/1024/1024/1024
        data_size = torch.cuda.max_memory_allocated('cuda:0')/1024/1024/1024
        y.append(data_size)
    except:
        print("最大长度序列为："+str(x[-1]))
        break
plt.plot(x,y)

In [4]:
# coeff = np.polyfit(x,y,2) 
# p=np.poly1d(coeff) 
# plt.plot(x, p(x)) 
# print(p)

In [4]:
model.eval()

MossForCausalLM(
  (transformer): MossModel(
    (wte): Embedding(107008, 6144)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-33): 34 x MossBlock(
        (ln_1): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
        (attn): MossAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (out_proj): QuantLinear()
          (qkv_proj): QuantLinear()
        )
        (mlp): MossMLP(
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
          (fc_in): QuantLinear()
          (fc_out): QuantLinear()
        )
      )
    )
    (ln_f): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=6144, out_features=107008, bias=True)
)